In [3]:
from itertools import chain
from nltk.corpus.reader import ConllCorpusReader

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics


In [1]:
#Estudar depois como usar o conll2003
with open('/home/82068895153/POS/skweak/data/Ontonotes/ner_train_1.txt', 'r') as file:
  sentences = list(file.readlines())

  print (sentences[5])

#  print (sentences[5])

be	O



In [33]:
def preprocess(sentences):
    l_sentences = []
    l1_ = []
    for token in sentences[5:]:
        cls = token.split()    
        if len(cls) != 0:
            l1_.append(cls)
        else:
            l_sentences.append(l1_)
            l1_ = []
    return l_sentences
        

In [ ]:
def preprocessO(sentences):
    l_sentences = []
    l1_ = []
    for token in sentences[5:]:
        cls = token.split()    
        if len(cls) != 0:
            l1_.append(cls)
        else:
            l_sentences.append(l1_)
            l1_ = []
    return l_sentences
        

In [34]:
sentences=preprocess(sentences)

In [53]:
print (sentences[5])

[['"', '"', 'O', 'O'], ['We', 'PRP', 'B-NP', 'O'], ['do', 'VBP', 'B-VP', 'O'], ["n't", 'RB', 'I-VP', 'O'], ['support', 'VB', 'I-VP', 'O'], ['any', 'DT', 'B-NP', 'O'], ['such', 'JJ', 'I-NP', 'O'], ['recommendation', 'NN', 'I-NP', 'O'], ['because', 'IN', 'B-SBAR', 'O'], ['we', 'PRP', 'B-NP', 'O'], ['do', 'VBP', 'B-VP', 'O'], ["n't", 'RB', 'I-VP', 'O'], ['see', 'VB', 'I-VP', 'O'], ['any', 'DT', 'B-NP', 'O'], ['grounds', 'NNS', 'I-NP', 'O'], ['for', 'IN', 'B-PP', 'O'], ['it', 'PRP', 'B-NP', 'O'], [',', ',', 'O', 'O'], ['"', '"', 'O', 'O'], ['the', 'DT', 'B-NP', 'O'], ['Commission', 'NNP', 'I-NP', 'B-ORG'], ["'s", 'POS', 'B-NP', 'O'], ['chief', 'JJ', 'I-NP', 'O'], ['spokesman', 'NN', 'I-NP', 'O'], ['Nikolaus', 'NNP', 'I-NP', 'B-PER'], ['van', 'NNP', 'I-NP', 'I-PER'], ['der', 'FW', 'I-NP', 'I-PER'], ['Pas', 'NNP', 'I-NP', 'I-PER'], ['told', 'VBD', 'B-VP', 'O'], ['a', 'DT', 'B-NP', 'O'], ['news', 'NN', 'I-NP', 'O'], ['briefing', 'NN', 'I-NP', 'O'], ['.', '.', 'O', 'O']]


In [51]:
#word 
sentences[5][1]


['We', 'PRP', 'B-NP', 'O']

In [52]:
#postag
sentences[5][2]

['do', 'VBP', 'B-VP', 'O']

In [27]:
def word2features(sent, i):
    
    word = sent[i][0]
    print('word', word)
    postag = sent[i][1]
    print('postag', postag)

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],        
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]        
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
                
    return features


In [41]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, __, label in sent]

def sent2tokens(sent):
    return [token for token, postag, __, label in sent]


In [54]:
X_train = [sent2features(s) for s in sentences]
#y_train = [sent2labels(s) for s in sentences]


ight
CD
fours
NNS
and
CC
three
CD
sixes
NNS
,
,
lifted
VBD
Surrey
NNP
to
TO
128
CD
for
IN
four
CD
on
IN
a
DT
rain-curtailed
JJ
day
NN
.
.
Fourth-placed
JJ
Leicestershire
NNP
had
VBD
Hampshire
NNP
on
IN
the
DT
ropes
NNS
at
IN
Leicester
NNP
before
IN
rain
NN
intervened
VBD
.
.
Pace
NNP
trio
NN
David
NNP
Millns
NNS
(
(
2-24
CD
)
)
,
,
Gordon
NNP
Parsons
NNP
(
(
3-20
CD
)
)
and
CC
Vince
NNP
Wells
NNP
(
(
2-19
CD
)
)
had
VBD
Hampshire
NNP
reeling
VBG
at
IN
81
CD
for
IN
seven
CD
in
IN
reply
NN
to
TO
the
DT
home
NN
county
NN
's
POS
first
JJ
innings
NN
of
IN
353
CD
.
.
Yorkshire
NNP
rekindled
VBN
their
PRP$
title
NN
hopes
NNS
after
IN
three
CD
successive
JJ
defeats
NNS
by
IN
taking
VBG
the
DT
upper
JJ
hand
NN
against
IN
arch-rivals
JJ
Lancashire
NNP
at
IN
Old
NNP
Trafford
NNP
.
.
Facing
VBG
Yorkshire
NNP
's
POS
529
CD
for
IN
eight
CD
declared
NN
,
,
Lancashire
NNP
were
VBD
forced
VBN
to
TO
follow
VB
on
IN
206
CD
behind
IN
after
IN
being
VBG
bowled
VBD
out
RP
for
IN
323
CD
,
,
paceman
NN
Darren

KeyboardInterrupt: 

In [11]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [12]:
labels = list(crf.classes_)
labels.remove('O')
labels

#PERSON == 'B-PER' 'I-PER' (CONLL), ORG == 'B-ORG', GPE == 'B-LOC' 'I-LOC' (CONLL), MISC == TUDO QUE NAO AS OUTRAS 3 NO CONLL

['B-MISC', 'B-PER', 'I-PER', 'B-LOC', 'B-ORG', 'I-ORG', 'I-MISC', 'I-LOC']

In [22]:
#Abre o Ontonotes 
with open('/home/82068895153/POS/skweak/data/Ontonotes/train_out.txt', 'r') as file:
    test_sentences = list(file.readlines())

print(test_sentences[20])

冲突



In [14]:
test_sentences = preprocess(test_sentences)

In [23]:
test_sentences[114]

'school\n'

In [24]:
X_test =  [sent2features(s) for s in test_sentences]

IndexError: string index out of range

In [16]:
#Aqui fiquei na duvida de quem seria o y_test
y_test = [sent2labels(s) for s in test_sentences]

ValueError: not enough values to unpack (expected 4, got 2)

In [17]:
y_pred = crf.predict(X_test)

In [18]:
y_test[2], y_pred[2]

NameError: name 'y_test' is not defined

In [24]:
metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

0.8026215689315441

In [ ]:
## carregar a base ontonotes

## fazer mapeamento das labels
## fazer a predição
## fazer a comparação entre y_predito e y_real